In [22]:
import re
import pandas as pd
import nltk
from collections import Counter

In [14]:
#nltk.download('punkt') 
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [15]:
def parse_document(document):
    document = re.sub('\n', ' ', document) #替換document中的換行符號為空格
    if isinstance(document, str):          #isinstance用來看資料型態
        document = document
    else:
        raise ValueError('Document is not string!') #發起錯誤
    document = document.strip()            #刪除開頭和結尾的空格
    sentences = nltk.sent_tokenize(document) #斷句
    sentences = [sentence.strip() for sentence in sentences]
    return sentences
# sample document

text = """
FIFA was founded in 1904 to oversee international competition among the national associations of Belgium, 
Denmark, France, Germany, the Netherlands, Spain, Sweden, and Switzerland. Headquartered in Zürich, its 
membership now comprises 211 national associations. Member countries must each also be members of one of 
the six regional confederations into which the world is divided: Africa, Asia, Europe, North & Central America 
and the Caribbean, Oceania, and South America.
"""

# tokenize sentences
sentences = parse_document(text)
tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]

In [17]:
sentences

['FIFA was founded in 1904 to oversee international competition among the national associations of Belgium,  Denmark, France, Germany, the Netherlands, Spain, Sweden, and Switzerland.',
 'Headquartered in Zürich, its  membership now comprises 211 national associations.',
 'Member countries must each also be members of one of  the six regional confederations into which the world is divided: Africa, Asia, Europe, North & Central America  and the Caribbean, Oceania, and South America.']

In [16]:
tokenized_sentences

[['FIFA',
  'was',
  'founded',
  'in',
  '1904',
  'to',
  'oversee',
  'international',
  'competition',
  'among',
  'the',
  'national',
  'associations',
  'of',
  'Belgium',
  ',',
  'Denmark',
  ',',
  'France',
  ',',
  'Germany',
  ',',
  'the',
  'Netherlands',
  ',',
  'Spain',
  ',',
  'Sweden',
  ',',
  'and',
  'Switzerland',
  '.'],
 ['Headquartered',
  'in',
  'Zürich',
  ',',
  'its',
  'membership',
  'now',
  'comprises',
  '211',
  'national',
  'associations',
  '.'],
 ['Member',
  'countries',
  'must',
  'each',
  'also',
  'be',
  'members',
  'of',
  'one',
  'of',
  'the',
  'six',
  'regional',
  'confederations',
  'into',
  'which',
  'the',
  'world',
  'is',
  'divided',
  ':',
  'Africa',
  ',',
  'Asia',
  ',',
  'Europe',
  ',',
  'North',
  '&',
  'Central',
  'America',
  'and',
  'the',
  'Caribbean',
  ',',
  'Oceania',
  ',',
  'and',
  'South',
  'America',
  '.']]

In [4]:
# tag sentences and use nltk's Named Entity Chunker
tagged_sentences = [nltk.pos_tag(sentence) for sentence in tokenized_sentences]  #nltk.pos_tag(sentence)詞性標記
ne_chunked_sents = [nltk.ne_chunk(tagged) for tagged in tagged_sentences]

# extract all named entities
named_entities = []
for ne_tagged_sentence in ne_chunked_sents:
    for tagged_tree in ne_tagged_sentence:
       # extract only chunks having NE labels
        if hasattr(tagged_tree, 'label'): #hasattr判斷tagged_tree是否有label屬性(是否有被標記)
            entity_name = ' '.join(c[0] for c in tagged_tree.leaves()) #get NE name
            entity_type = tagged_tree.label() # get NE category
            named_entities.append((entity_name, entity_type))
            # get unique named entities
            named_entities = list(set(named_entities))

# store named entities in a data frame
entity_frame = pd.DataFrame(named_entities, columns=['Entity Name', 'Entity Type'])
# display results
print(entity_frame)

        Entity Name   Entity Type
0            France           GPE
1            Africa        PERSON
2           Denmark           GPE
3             Spain           GPE
4            Sweden           GPE
5       Switzerland           GPE
6            Zürich           GPE
7         Caribbean      LOCATION
8             North           GPE
9     South America           GPE
10          Germany           GPE
11      Netherlands           GPE
12             Asia           GPE
13           Europe           GPE
14             FIFA  ORGANIZATION
15          Belgium           GPE
16  Central America  ORGANIZATION
17          Oceania           GPE


# 匯入twitter貼文

In [18]:
tweets = pd.read_csv('/Users/Rebeca/Downloads/道瓊twitter(1).csv')

In [19]:
tweets.head()

,id,created_at,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,possibly_sensitive,hashtags,user_mentions,place,place_coord_boundaries
0,1252214966691811328,Mon Apr 20 12:38:07 +0000 2020,"<a href=""https://mobile.twitter.com"" rel=""nofo...","@thehill Not so fast, my friend—that little ol...",thehill Not fast friend—that little ol ’ Dow J...,"Sentiment(polarity=-0.14583333333333334, subje...",-0.14583333333333334,0.5,en,0,0,HenryLWilson1,NaN,NaN,thehill,East TN holler in the Smokies,NaN
1,1252212775289589760,Mon Apr 20 12:29:25 +0000 2020,"<a href=""http://twitter.com/download/android"" ...",RT @Investingcom: 🚨 Wall Street Performance Si...,RT Investingcom 🚨 Wall Street Performance Sinc...,"Sentiment(polarity=-0.15, subjectivity=0.39999...",-0.15,0.39999999999999997,en,0,27,coolnittu,NaN,NaN,Investingcom,India,NaN
2,1252207578689536003,Mon Apr 20 12:08:46 +0000 2020,"<a href=""https://mobile.twitter.com"" rel=""nofo...","RT @elerianm: From the @WSJ\n""The Dow Jones In...",RT elerianm From WSJ '' The Dow Jones Industri...,"Sentiment(polarity=0.1388888888888889, subject...",0.1388888888888889,0.4333333333333333,en,0,137,NutitCZ,NaN,NaN,"elerianm, WSJ","Prague, Czech Republic",NaN
3,1252205461681692677,Mon Apr 20 12:00:21 +0000 2020,"<a href=""http://twitter.com/download/android"" ...",RT @Investingcom: 🚨 Wall Street Performance Si...,RT Investingcom 🚨 Wall Street Performance Sinc...,"Sentiment(polarity=-0.15, subjectivity=0.39999...",-0.15,0.39999999999999997,en,0,27,BetoMendeleiev,NaN,NaN,Investingcom,Capital Federal,NaN
4,1252205072840351752,Mon Apr 20 11:58:48 +0000 2020,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",RT @Investingcom: 🚨 Wall Street Performance Si...,RT Investingcom 🚨 Wall Street Performance Sinc...,"Sentiment(polarity=-0.15, subjectivity=0.39999...",-0.15,0.39999999999999997,en,0,27,Investingcom,NaN,NaN,Investingcom,Global,NaN


In [20]:
# tokenize sentences
tokenized_sentences = []
for i in tweets['original_text']:
    sentences = parse_document(i)
    for sentence in sentences:
        tokenized_sentences.append(nltk.word_tokenize(sentence) )

In [10]:
tagged_sentences = [nltk.pos_tag(sentence) for sentence in tokenized_sentences]  #nltk.pos_tag(sentence)詞性標記
ne_chunked_sents = [nltk.ne_chunk(tagged) for tagged in tagged_sentences]

In [11]:
ne_chunked_sents

[Tree('S', [('@', 'NNS'), ('thehill', 'MD'), ('Not', 'RB'), ('so', 'RB'), ('fast', 'RB'), (',', ','), ('my', 'PRP$'), ('friend—that', 'JJ'), ('little', 'JJ'), ('ol', 'JJ'), ('’', 'NN'), Tree('PERSON', [('Dow', 'NNP'), ('Jones', 'NNP'), ('Industrial', 'NNP'), ('Average', 'NNP')]), ('will', 'MD'), ('just', 'RB'), ('have', 'VB'), ('to', 'TO'), ('take', 'VB'), ('its', 'PRP$'), ('lumps', 'NNS'), ('like', 'IN'), ('the', 'DT'), ('rest', 'NN'), ('of', 'IN'), ('us', 'PRP')]),
 Tree('S', [('RT', 'NNP'), ('@', 'NNP'), ('Investingcom', 'NNP'), (':', ':'), ('🚨', 'NN'), Tree('FACILITY', [('Wall', 'NNP'), ('Street', 'NNP'), ('Performance', 'NNP')]), ('Since', 'IN'), ('January', 'NNP'), ('1', 'CD'), (',', ','), ('2020', 'CD'), ('🚨', 'NN'), ('❌Dow', 'NNP'), Tree('PERSON', [('Jones', 'NNP'), ('Industrial', 'NNP'), ('Average', 'NNP')]), ('⬇️', 'VBD'), ('-15', 'NNP'), ('%', 'NN'), ('❌S', 'NNP'), ('&', 'CC'), ('amp', 'NN'), (';', ':'), ('P', 'NNP'), ('500', 'CD'), ('Index', 'NNP'), ('⬇️', 'NNP'), ('-11', '

In [12]:
# extract all named entities
named_entities = []
for ne_tagged_sentence in ne_chunked_sents:
    for tagged_tree in ne_tagged_sentence:
       # extract only chunks having NE labels
        if hasattr(tagged_tree, 'label'): #hasattr判斷tagged_tree是否有label屬性(是否有被標記)
            entity_name = ' '.join(c[0] for c in tagged_tree.leaves()) #get NE name
            entity_type = tagged_tree.label() # get NE category
            named_entities.append((entity_name, entity_type))
            # get unique named entities
            named_entities = list(set(named_entities))

# store named entities in a data frame
entity_frame = pd.DataFrame(named_entities, columns=['Entity Name', 'Entity Type'])
# display results
print(entity_frame)

                                 Entity Name   Entity Type
0                             DerangedDonald  ORGANIZATION
1                                        Dow  ORGANIZATION
2                                        SPY  ORGANIZATION
3                            SenatorLoeffler  ORGANIZATION
4                                     United  ORGANIZATION
5                                FTSE Nareit  ORGANIZATION
6                                   New York           GPE
7                      DowJones Stock Market  ORGANIZATION
8                                      COVID  ORGANIZATION
9                                    Bitcoin        PERSON
10                                   RepTrey  ORGANIZATION
11                                     Rally        PERSON
12                                   Analyst        PERSON
13                                    Slides  ORGANIZATION
14                              Global Cases        PERSON
15                             Recent Jargon  ORGANIZATI

In [ ]:
#保留有意義的字詞
remain_word=[]
for i in entity_frame['Entity Name']:
    remain_word.append(i)

In [98]:
remain_word

['DerangedDonald',
 'Dow',
 'SPY',
 'SenatorLoeffler',
 'United',
 'FTSE Nareit',
 'New York',
 'DowJones Stock Market',
 'COVID',
 'Bitcoin',
 'RepTrey',
 'Rally',
 'Analyst',
 'Slides',
 'Global Cases',
 'Recent Jargon',
 'Market Update',
 'Dow',
 'CNBC Dow Jones Industrial Average',
 'Houston',
 'Noticia',
 'SPDR',
 'WallStreet',
 'MSCI',
 'Coronavirus',
 'Dow Continues Declines Off',
 'US30',
 'MSpicuzzaMJS Weird',
 'ShandaPanda333',
 'NEW',
 'Dow Jones Industrial Average Index',
 'No One Really',
 'MARKETWATCH Futures',
 'Click',
 'realDonaldTrump',
 'AndyOstroy No',
 'Dow',
 'Stocks Eke Out Gains',
 'Dow Jones Industrial Average',
 'Stocks',
 'Average',
 'Bitcoin',
 'Good',
 'Neil_Irwin Dow Jones Industrial',
 'NYSE Dow Jones Industrial Average',
 'Earnings Season Begins',
 'Trumka',
 'Hey',
 'U.S.A.',
 'Electric Launched',
 'History',
 'Gilead Sciences',
 'U.S. Economy',
 'Latest Grim Economic News',
 'DIA',
 'WINNING',
 'STAT',
 'SethAbramson',
 'SILENT',
 'Chicago',
 'Virus',


# 嘗試全部貼文的斷詞包成一個list，去除stopwords

In [24]:
# tokenize sentences
tokenized_sentences = []
for i in tweets['original_text']:
    sentences = parse_document(i)
    for sentence in sentences:
        for words in nltk.word_tokenize(sentence):
            tokenized_sentences.append(words)

In [26]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [27]:
words = []
for w in tokenized_sentences:
    if w not in stopwords:
        words.append(w)

In [30]:
Counter(words)

Counter({'@': 25785,
         'thehill': 57,
         'Not': 79,
         'fast': 21,
         ',': 15696,
         'friend—that': 21,
         'little': 21,
         'ol': 21,
         '’': 1148,
         'Dow': 11907,
         'Jones': 11992,
         'Industrial': 11373,
         'Average': 10898,
         'take': 33,
         'lumps': 21,
         'like': 225,
         'rest': 42,
         'us': 47,
         'RT': 16494,
         'Investingcom': 496,
         ':': 26561,
         '🚨': 988,
         'Wall': 745,
         'Street': 717,
         'Performance': 494,
         'Since': 1064,
         'January': 475,
         '1': 974,
         '2020': 738,
         '❌Dow': 494,
         '⬇️': 988,
         '-15': 475,
         '%': 3168,
         '❌S': 494,
         '&': 2787,
         'amp': 2667,
         ';': 3297,
         'P': 875,
         '500': 870,
         'Index': 698,
         '-11': 475,
         '❌NASDAQ': 475,
         'C…': 456,
         'elerianm': 2545,
         'From'

In [40]:
names = {}          
relationships = {}  
lineNames = []  
for i in tweets['original_text']:
    try:
        sentences = parse_document(i)
        for sentence in sentences:        
            poss = nltk.word_tokenize(sentence)
            lineNames.append([])
            for w in poss:
                if w not in stopwords:
                    lineNames[-1].append(w)   #每一篇文章一個list     
                if names.get(w) is None and w not in stopwords:    
                    relationships[w] = {}            
    except:
        print(i)

In [47]:
term_dic = dict()
for sentence in lineNames:
    for term in sentence:
        if term not in term_dic:
            term_dic[term] = {}

In [48]:
for i in range(len(lineNames)):
    for term1 in term_dic:
        num = 0
        for term2 in lineNames[i]:
            if term1 == term2:
                num += 1
        term_dic[term1][tweets['original_text'][i]] = num

KeyError: 21968

In [51]:
term_dic.keys()

dict_keys(['@', 'thehill', 'Not', 'fast', ',', 'friend—that', 'little', 'ol', '’', 'Dow', 'Jones', 'Industrial', 'Average', 'take', 'lumps', 'like', 'rest', 'us', 'RT', 'Investingcom', ':', '🚨', 'Wall', 'Street', 'Performance', 'Since', 'January', '1', '2020', '❌Dow', '⬇️', '-15', '%', '❌S', '&', 'amp', ';', 'P', '500', 'Index', '-11', '❌NASDAQ', 'C…', 'elerianm', 'From', 'WSJ', '``', 'The', 'staged', 'best', 'two-week', 'performance', 'since', '1930s', 'dramatic', 'rebound', 'that…', 'Gold', 'Price', 'vs', 'Stock', 'Market', '-', '100', 'Year', 'Chart', 'compares', 'historical', 'percentage', 'return', 'Indust…', 'https', '//t.co/QE0sLmI7Rr', 'Futures', 'tied', 'fell', '1.7', 'suggesting', 'blue-chips', 'gauge', 'may', 'drop', '300…', '//t.co/4VyVAx8hLM', 'In', 'years', 'priced', 'ounces', 'gold', 'gone', 'nowhere', '.', 'However', 'would', 'one', 'sw…', '//t.co/sLKBW9vXKM', 'A', 'sharp', 'decline', 'U.S.', 'crude', 'prices', 'futures', '392', 'points', 'pointing', 'Monday', 'o…', '//

In [52]:
term_dic['@']

{'@thehill Not so fast, my friend—that little ol’ Dow Jones Industrial Average will just have to take its lumps like the rest of us': 0,
 'RT @Investingcom: 🚨 Wall Street Performance Since January 1, 2020 🚨\n\n❌Dow Jones Industrial Average ⬇️ -15%\n❌S&amp;P 500 Index ⬇️ -11%\n❌NASDAQ C…': 0,
 'RT @elerianm: From the @WSJ\n"The Dow Jones Industrial Average staged its best two-week performance since the 1930s, a dramatic rebound that…': 2,
 'Gold Price vs Stock Market - 100 Year Chart that compares the historical percentage return for the Dow Jones Indust… https://t.co/QE0sLmI7Rr': 2,
 'Futures tied to the Dow Jones Industrial Average fell 1.7%, suggesting that the blue-chips gauge may drop about 300… https://t.co/4VyVAx8hLM': 0,
 'In 100 years the Dow Jones Industrial Average priced in ounces of gold have gone nowhere. However would one have sw… https://t.co/sLKBW9vXKM': 0,
 'A sharp decline in U.S. crude prices.\xa0\nDow Jones Industrial Average futures\xa0fell 392 points, pointing to 

In [53]:
lineNames[0]

['@',
 'thehill',
 'Not',
 'fast',
 ',',
 'friend—that',
 'little',
 'ol',
 '’',
 'Dow',
 'Jones',
 'Industrial',
 'Average',
 'take',
 'lumps',
 'like',
 'rest',
 'us']

In [57]:
for i in term_dic:
    b = i
    print(i)
    print(i == lineNames[0][0])
    break

@
True


In [58]:
for i in lineNames[0]:
    a = i
    print(i)
    break

@


In [59]:
a == b

True

In [60]:
tweets['original_text'][0]

'@thehill Not so fast, my friend—that little ol’ Dow Jones Industrial Average will just have to take its lumps like the rest of us'

In [61]:
lineNames[0]

['@',
 'thehill',
 'Not',
 'fast',
 ',',
 'friend—that',
 'little',
 'ol',
 '’',
 'Dow',
 'Jones',
 'Industrial',
 'Average',
 'take',
 'lumps',
 'like',
 'rest',
 'us']

In [62]:
len(term_dic)

2564

In [73]:
print(tweets['clean_text'][6])

Futures tied Dow Jones Industrial Average fell 1.7 suggesting blue-chips gauge may drop 300… https //t.co/4VyVAx8hLM


# 試試stanza

In [74]:
import stanza
from stanza.utils.resources import DEFAULT_MODEL_DIR

In [75]:
stanza.download('en')

2020-04-25 17:19:43 INFO: Downloading default packages for language: en (English)...
2020-04-25 17:44:15 INFO: Finished downloading models and saved to /Users/Rebeca/stanza_resources.


In [88]:
en_nlp = stanza.Pipeline(lang = 'zh-hant', processors = 'tokenize,lemma,pos',use_gpu = False)

2020-04-26 11:46:56 INFO: Loading these models for language: zh-hant (Traditional_Chinese):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| pos       | gsd     |
| lemma     | gsd     |

2020-04-26 11:46:56 INFO: Use device: cpu
2020-04-26 11:46:56 INFO: Loading: tokenize
2020-04-26 11:46:56 INFO: Loading: pos
2020-04-26 11:46:57 INFO: Loading: lemma
2020-04-26 11:46:57 INFO: Done loading processors!


In [87]:
segment=[]
lemma = []
for num in range(len(tweets['original_text'])):
    for i in en_nlp(tweets['original_text'][num]).sentences[0].words:
        segment.append(i.text)
        lemma.append(i.lemma)


In [90]:
len(segment)

456836

In [93]:
dic = {'segment':segment,'lemma':lemma}
df = pd.DataFrame(dic)

,segment,lemma
0,@thehill,@thehill
1,Not,Not
2,so,so
3,"fast,","fast,"
4,my,my
5,friend—that,friend—that
6,little,little
7,ol’,ol’
8,Dow,Dow
9,Jones,Jones


# 把斷詞後的字詞只留下有意義的字且算次數

In [100]:
last=[]
for i in segment:
    if i in remain_word:
        last.append(i)

In [102]:
len(last)

43879

In [103]:
last

['Dow',
 'Jones',
 'Average',
 'Wall',
 'Street',
 'Dow',
 'Jones',
 'Average',
 'Wall',
 'Street',
 'Wall',
 'Street',
 'Gold',
 'Stock',
 'Market',
 'Dow',
 'Jones',
 'Dow',
 'Jones',
 'Average',
 'Dow',
 'Jones',
 'Average',
 'U.S.',
 'Dow',
 'Jones',
 'Average',
 'Dow',
 'Jones',
 'Average',
 'US30',
 'Dow',
 'Jones',
 'Average',
 'Dow',
 'Jones',
 'Average',
 'Dow',
 'Jones',
 'Average',
 'Dow',
 'Jones',
 'US',
 'Dow',
 'Jones',
 'Average',
 'Dow',
 'Jones',
 'March',
 'Dow',
 'Jones',
 'Average',
 'GLOBAL',
 'INDICES',
 'US',
 'Composite',
 'Dow',
 'Jones',
 'Average',
 'Dow',
 'Jones',
 'Average',
 'Dow',
 'Jones',
 'Average',
 'US',
 'Dow',
 'Jones',
 'Average',
 'US',
 'Dow',
 'Jones',
 'Average',
 'US',
 'Dow',
 'Jones',
 'Average',
 'US',
 'Dow',
 'Jones',
 'Average',
 'Dow',
 'Jones',
 'Disruption',
 'Dow',
 'Jones',
 'Average',
 'Dow',
 'Jones',
 'Average',
 'Dow',
 'Jones',
 'Average',
 'Inauguration',
 'Dow',
 'Jones',
 'U.S.',
 'Dow',
 'Jones',
 'Average',
 'SPDR',
 'D